In [16]:
import breeze.linalg._
import scala.io.Source
import breeze.signal._

In [17]:
// take in #2 PC1 score (from Apache Spark SVD on entire #2 with 4 PCs)
// as material for noise profile
val fileName = "data/#2_PC1_score/part-00000"

fileName = data/#2_PC1_score/part-00000


data/#2_PC1_score/part-00000

In [18]:
%%script false
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.11 --transitive

Name: Error parsing magics!
Message: Magic script does not exist!
StackTrace: 

In [19]:
import scala.math.abs
import vegas._

val experimentName = "#2"

val noiseSegments = {
 
val samplingRate = 10000
val secondStart = 0
val secondEnd = 30
val indexUntil = 10000

val indexStart = samplingRate * secondStart
val indexEnd = samplingRate * secondEnd

// https://alvinalexander.com/source-code/scala-function-read-text-file-into-array-list
def readFile(fileName: String, indexStart: Int, indexEnd: Int): Array[Double] = {
    val bufferedSource = Source.fromFile(fileName)
    val lines = (for (line <- bufferedSource
                      .getLines
                      //.slice(indexStart, indexEnd)
                      //.take(indexUntil)
                      .map(_.toDouble)) yield line).toArray
    bufferedSource.close
    lines
}
val data = readFile(fileName, indexStart, indexEnd)
val maxValue = Array(abs(data.max), abs(data.min)).max
val totalLength = data.length

/*    
// Prepare loadings for plotting
// with Vegas
var inputDataResult = data.toArray.zipWithIndex.map{ case (e, i) =>
        Map("Seconds" -> i/samplingRate.toDouble, 
            "Magnitude" -> e )
        }
        .toArray

// Plot Loadings with Vegas
Vegas("Score1 from raw data of " + experimentName, width=1000, height=500)
    .withData(inputDataResult)
    .mark(Line)
    .encodeY("Magnitude", Quantitative)
    .encodeX("Seconds", Quantitative)
    .show
*/
    
// extract noise segments

val fractionNoiseOfMaxValue = 0.27

val noise = for {
    (v, i) <- data.zipWithIndex
    if (i < totalLength - samplingRate) && data.slice(i, i + samplingRate).forall(v => v < fractionNoiseOfMaxValue * maxValue)
} yield (v, i)

/*    
// Prepare loadings for plotting
// with Vegas
var noisePrepared = noise.map{ case (e, i) =>
        Map("Seconds" -> i/samplingRate.toDouble, 
            "Magnitude" -> e )
        }
        .toArray
// Plot Loadings with Vegas
Vegas("Score1 LP from raw data of " + experimentName, width=1000, height=500)
    .withData(noisePrepared)
    .mark(Line)
    .encodeY("Magnitude", Quantitative)
    .encodeX("Seconds", Quantitative)
    .show
*/
val folderName = experimentName + "_PC1_score_noise_segments"
sc.parallelize(noise).coalesce(1).saveAsTextFile("data/" + folderName)
}


experimentName = #2


noiseSegments: Unit = ()


#2